# Перенос стратегии на другие строки

Этот ноутбук позволяет:
1. Прочитать исходный TVR файл
2. Указать базовые строки стратегии (которые хотим перенести)
3. Просмотреть и изменить порядок строк
4. Указать новую базовую строку (лонг)
5. Получить новый TVR с перенесенной стратегией


## Шаг 0: Инициализация


In [1]:
import sys
from pathlib import Path
import pandas as pd

# Добавляем корень проекта в путь
project_root = Path.cwd().resolve()
if not (project_root / 'src').exists():
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Корень проекта: {project_root}")


Корень проекта: C:\Users\user\Documents\piranha\constructor_TVR


In [2]:
from src.tvr_service.io.tvr_io import read_tvr2, write_tvr2
from src.tvr_service.generator import (
    build_layout_from_source,
    generate_robot_segment,
)
from converters import tvr2_to_excel, TVRParams

print("Модули загружены успешно")


Модули загружены успешно


## Шаг 1: Загрузка исходного TVR файла

**Что нужно указать:**
- `source_tvr_path` — путь к исходному TVR файлу


In [3]:
# Укажите путь к исходному TVR файлу
source_tvr_path = project_root / 'Default.tvr2'

# Загружаем файл
tvr_file = read_tvr2(source_tvr_path)
tvr_df = tvr_file.dataframe

print(f"✓ Загружен TVR: {source_tvr_path.name}")
print(f"  Всего строк: {len(tvr_df)}")
print(f"  Столбцы: {list(tvr_df.columns)}")
print(f"\nПервые несколько строк:")
tvr_df.head(10)


✓ Загружен TVR: Default.tvr2
  Всего строк: 6511
  Столбцы: ['stroka', 'Start', 'Kill all', 'Out only', 'InL1', 'InL2', 'OutL1', 'OutL2', 'Pos', 'Limit', 'MFixS', 'Sec 0', 'V 0', 'Sec 1', 'V 1', 'W 1', 'Sec 2', 'V 2', 'W 2', 'Sec 3', 'V 3', 'W 3', 'Sec 4', 'V 4', 'W 4', 'Sec 5', 'V 5', 'W 5', 'Sec 6', 'V 6', 'W 6', 'Sec 7', 'V 7', 'W 7', 'Sec 8', 'V 8', 'W 8', 'Sec 9', 'V 9', 'W 9', 'Sec 10', 'V 10', 'W 10', 'Sec 11', 'V 11', 'W 11', 'Sec 12', 'V 12', 'W 12', '[k]', '[a]', '[b]', '[n]', 'C', 'N', 'P', 'A', 'C1', 'E', 'S', 'SM', 'SrSv', 'MBlock', 'Mode', 'St.D', 'T.In', 'T.Out', 'NotSet', 'FrId', 'MoveN', 'secIn', 'secOut', 'XN', 'MP', 'xx']

Первые несколько строк:


stolbec,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,Limit,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
0,1,NaN,NaN,NaN,█████,█████,█████,█████,█████,NaN,...,█████,█████,█████,█████,█████,█████,█████,█████,█████,█████
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28,NaN,NaN,NaN,█████,█████,█████,█████,█████,NaN,...,█████,█████,█████,█████,█████,█████,█████,█████,█████,█████
3,29,NaN,NaN,NaN,31-170,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30,NaN,NaN,NaN,31-32,█████,█████,█████,█████,NaN,...,█████,█████,█████,█████,█████,█████,█████,█████,█████,█████
5,31,True,NaN,NaN,33,NaN,33,NaN,NaN,NaN,...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,NaN,3/1,3/3,NaN,NaN,0.005,NaN,NaN
6,32,True,NaN,NaN,33,NaN,33,NaN,NaN,NaN,...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,NaN,3/1,3/3,NaN,NaN,0.005,NaN,NaN
7,33,NaN,NaN,NaN,34,NaN,34,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,3/1,NaN,NaN,NaN,NaN,NaN,NaN
8,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,09:00:00-23:50:00,09:00:00-23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,40,NaN,NaN,NaN,41-42,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Опционально: Сохраним TVR в Excel для просмотра


In [ ]:
# Сохраняем в Excel для удобного просмотра
snapshot_path = project_root / 'docs' / 'transfer_snapshot.xlsx'
tvr_df.to_excel(snapshot_path, index=False)
print(f"✓ Сохранено в Excel: {snapshot_path}")


## Шаг 2: Указание базовых строк стратегии

**Что нужно указать:**
- `base_strokas` — список номеров строк (stroka), которые составляют вашу стратегию
  - Обычно это строки лонг и шорт позиций
  - Можно указать одну или несколько строк


In [4]:
# Укажите номера базовых строк стратегии
# Пример: строки лонг и шорт
base_strokas = [211, 212]

print(f"Выбраны базовые строки: {base_strokas}")
print("\nДанные выбранных строк:")
selected_rows = tvr_df[tvr_df['stroka'].isin(base_strokas)]
selected_rows


Выбраны базовые строки: [211, 212]

Данные выбранных строк:


stolbec,stroka,Start,Kill all,Out only,InL1,InL2,OutL1,OutL2,Pos,Limit,...,T.In,T.Out,NotSet,FrId,MoveN,secIn,secOut,XN,MP,xx
89,211,True,NaN,NaN,213,NaN,213,NaN,1;0;0;,NaN,...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,NaN,5/1,3/3,NaN,NaN,0.005,NaN,NaN
90,212,True,NaN,NaN,213,NaN,213,NaN,NaN,NaN,...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,09:00:20-10:00:00 10:01:00-11:00:00 11:01:00-1...,NaN,5/1,3/3,NaN,NaN,0.005,NaN,NaN


## Шаг 3: Парсинг структуры и просмотр связей

Система автоматически найдет все связанные строки (фильтры, индикаторы и т.д.)


In [5]:
# Строим структуру (layout) из выбранных строк
layout = build_layout_from_source(
    str(source_tvr_path),
    base_strokas,
    duplicate_shared_filters=False  # True - если хотите дублировать общие фильтры
)

print(f"✓ Структура построена")
print(f"  Базовая строка: {layout.primary_base_stroka}")
print(f"  Всего элементов: {len(layout.entries)}")
print(f"\nСтруктура стратегии (относительные смещения от базы {layout.primary_base_stroka}):")
print("=" * 60)
for entry in layout.entries:
    print(f"{entry.display:40s} исходная={entry.original_stroka:5d}  смещение={entry.relative_offset:+4d}")


✓ Структура построена
  Базовая строка: 211
  Всего элементов: 7

Структура стратегии (относительные смещения от базы 211):
#base_long                               исходная=  211  смещение=  +0
##filter_1_long_&_short                  исходная=  213  смещение=  +2
###filter_2_long_&_short                 исходная=  214  смещение=  +3
####filter_3_long_&_short                исходная=  215  смещение=  +4
#####filter_4_long_&_short               исходная=  216  смещение=  +5
######filter_5_long_&_short              исходная=  217  смещение=  +6
#base_short                              исходная=  212  смещение=  +1


## Шаг 4: Изменение порядка строк (опционально)

Если вы хотите изменить относительное расположение элементов стратегии:
1. Скопируйте вывод выше
2. Измените смещения (числа после `:`)
3. Раскомментируйте и выполните код ниже

**Формат:** `#имя: текущее_смещение : новое_смещение`


In [6]:
# Пример: если хотите изменить смещения, отредактируйте текст ниже
# Формат: #имя: текущее_смещение : новое_смещение

# edited_layout_text = """
# #MME55_long: 0 : 0
# ##filter_1_InM: -1 : -1
# ##filter_1_OutM: -2 : -2
# #MME55_short: 1 : 1
# ##filter_1_InM: -1 : -1
# ##filter_1_OutM: -2 : -2
# """

# from src.tvr_service.generator.layout import parse_layout_text
# layout_edits = parse_layout_text(layout, edited_layout_text)
# print("✓ Изменения применены")
# print(f"  Изменено смещений: {len(layout_edits.relative_overrides)}")

# Если не меняете порядок - создаем пустые правки
from src.tvr_service.generator.layout import LayoutEdits
layout_edits = LayoutEdits(relative_overrides={})
print("Порядок строк не изменен")


Порядок строк не изменен


## Шаг 5: Указание новой базовой строки

**Что нужно указать:**
- `new_base_stroka` — номер новой базовой строки (куда переносим стратегию)
  - Это будет новая позиция для "лонг" строки
  - Все остальные элементы будут расположены относительно этой строки
- `new_sec0_value` — значение для столбца "Sec 0" (тикер/инструмент)


In [ ]:
# Укажите новую базовую строку (куда переносим стратегию)
new_base_stroka = 961

# Укажите новое значение для "Sec 0" (тикер)
new_sec0_value = "NEW_TICKER"

print(f"✓ Новая базовая строка: {new_base_stroka}")
print(f"✓ Новый тикер: {new_sec0_value}")


✓ Новая базовая строка: 961
✓ Новый тикер: NEW_TICKER


## Шаг 6: Компиляция и подготовка к генерации


In [ ]:
from src.tvr_service.generator.layout import compile_layout

# Создаем mapping: старая база -> новая база
base_assignment = {layout.primary_base_stroka: new_base_stroka}

# Компилируем layout с учетом изменений
compiled = compile_layout(layout, layout_edits, base_assignment)

print("✓ Layout скомпилирован")
print(f"  Базовых строк: {len(compiled.base_assignment)}")
print(f"  Переопределений: {len(compiled.stroka_overrides)}")
print("\nИтоговое распределение строк:")
print("=" * 60)
all_mappings = {**compiled.base_assignment, **compiled.stroka_overrides}
for entry in layout.entries:
    old_stroka = entry.original_stroka
    new_stroka = all_mappings.get(old_stroka, new_base_stroka + entry.relative_offset)
    print(f"{entry.display:40s} {old_stroka:5d} → {new_stroka:5d}")


## Шаг 7: Генерация нового TVR

Создаем новый TVR файл с перенесенной стратегией


In [ ]:
# Пути для выходных файлов
output_tvr_path = project_root / f'generated_transfer_{new_sec0_value}.tvr2'
output_excel_path = project_root / f'generated_transfer_{new_sec0_value}.xlsx'

# Генерируем новый TVR
result = generate_robot_segment(
    source_tvr_path=str(source_tvr_path),
    base_strokas=base_strokas,
    base_assignment=compiled.base_assignment,
    sec0_value=new_sec0_value,
    output_tvr_path=str(output_tvr_path),
    output_excel_path=str(output_excel_path),
    duplicate_shared_filters=False,
    stroka_overrides=compiled.stroka_overrides,
)

print("✅ Новый TVR успешно создан!")
print(f"\n📄 TVR файл: {result.tvr_file}")
print(f"📊 Excel файл: {result.excel_file}")
print(f"\nПеренесено строк: {len(result.stroka_mapping)}")
print(f"Диапазон новых строк: {min(result.stroka_mapping.values())} - {max(result.stroka_mapping.values())}")


## Шаг 8: Проверка результата


In [ ]:
# Загружаем результат для проверки
result_df = result.dataframe

print("Результат генерации:")
print(f"  Строк в новом TVR: {len(result_df)}")
print(f"\nПервые строки:")
result_df.head(10)


In [ ]:
# Проверяем значение Sec 0
sec0_values = result_df['Sec 0'].unique()
print(f"Значения 'Sec 0' в результате: {sec0_values}")

# Проверяем базовые строки
base_rows = result_df[result_df['stroka'] == new_base_stroka]
if not base_rows.empty:
    print(f"\n✓ Базовая строка {new_base_stroka} найдена:")
    display(base_rows)
else:
    print(f"\n⚠ Базовая строка {new_base_stroka} не найдена")


## Шаг 9: Сравнение структуры (опционально)


In [ ]:
# Показываем mapping старых и новых строк
mapping_df = pd.DataFrame([
    {
        'старая_stroka': old,
        'новая_stroka': new,
        'смещение_от_базы': new - new_base_stroka
    }
    for old, new in sorted(result.stroka_mapping.items(), key=lambda x: x[1])
])

print("Полная таблица соответствия строк:")
mapping_df


## Готово! 🎉

Ваша стратегия успешно перенесена на новые строки.

**Следующие шаги:**
1. Откройте Excel файл для визуальной проверки
2. Проверьте все ссылки на фильтры и индикаторы
3. При необходимости повторите процесс с другими настройками
4. Используйте созданный TVR файл в вашей торговой системе


---

## Дополнительные инструменты

### Пакетный перенос на несколько инструментов


In [ ]:
# Если нужно перенести стратегию на несколько инструментов сразу

# Список инструментов и их базовых строк
instruments = [
    {'ticker': 'TICKER1', 'base': 5000},
    {'ticker': 'TICKER2', 'base': 6000},
    {'ticker': 'TICKER3', 'base': 7000},
]

# Раскомментируйте для выполнения пакетного переноса
# for instrument in instruments:
#     ticker = instrument['ticker']
#     base = instrument['base']
#     
#     output_tvr = project_root / f'generated_transfer_{ticker}.tvr2'
#     output_excel = project_root / f'generated_transfer_{ticker}.xlsx'
#     
#     base_assignment = {layout.primary_base_stroka: base}
#     compiled = compile_layout(layout, layout_edits, base_assignment)
#     
#     result = generate_robot_segment(
#         source_tvr_path=str(source_tvr_path),
#         base_strokas=base_strokas,
#         base_assignment=compiled.base_assignment,
#         sec0_value=ticker,
#         output_tvr_path=str(output_tvr),
#         output_excel_path=str(output_excel),
#         duplicate_shared_filters=False,
#         stroka_overrides=compiled.stroka_overrides,
#     )
#     
#     print(f"✓ {ticker}: {output_tvr.name}")
# 
# print("\n✅ Пакетный перенос завершен!")
